In [1]:
!pip install neo4j

In [10]:
from neo4j import GraphDatabase

neo4j_uri = "bolt://neo4j:7687"  # Using the forwarded Bolt port
neo4j_username = "neo4j"
neo4j_password = ""  # No password, as configured

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))

def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Hello, Neo4j!' AS greeting")
        for record in result:
            print(record["greeting"])

test_connection()


Hello, Neo4j!


In [11]:
with driver.session() as session:
    # First, create Genre nodes
    for genre_name in genres_list:
        session.run(
            "MERGE (g:Genre {name: $name})",
            {"name": genre_name}
        )
    # Next, create Movie and Actor nodes, and relationships
    for genre_name, movies in top5_movies_by_genre.items():
        for movie in movies:  # movie contains title, year, popularity
            session.run(
                """
                MERGE (m:Movie {title: $title})
                ON CREATE SET m.release_year = $year, m.popularity = $pop
                MERGE (g:Genre {name: $genre})
                MERGE (m)-[:HAS_GENRE]->(g)
                """,
                {
                    "title": movie.title,
                    "year": movie.release_year,
                    "pop": movie.popularity,
                    "genre": genre_name
                }
            )
            # Create Actor nodes and ACTED_IN relationships for this movie
            for actor in movie.actors:  # each actor has name, popularity
                session.run(
                    """
                    MERGE (a:Actor {name: $name})
                    ON CREATE SET a.popularity = $pop
                    MERGE (a)-[:ACTED_IN]->(m)
                    """,
                    {"name": actor.name, "pop": actor.popularity}
                )
    # Finally, create COACTED_WITH relationships between actors of the same movie
    for movie in all_top_movies:
        actors = movie.actors
        # For each unique pair of actors in this movie:
        for i in range(len(actors)):
            for j in range(i+1, len(actors)):
                session.run(
                    """
                    MATCH (a:Actor {name: $name1}), (b:Actor {name: $name2})
                    MERGE (a)-[:COACTED_WITH]-(b)
                    """,
                    {"name1": actors[i].name, "name2": actors[j].name}
                )


NameError: name 'genres_list' is not defined